<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr1/blob/main/Gavrilkov%20Nikolay/internship_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1-я стажировка "Профпатолог"

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer

# Дополнительные инструменты
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 

import random 
import math 
import os 
import re 
%matplotlib inline

In [2]:
df_stock = pd.read_excel('/content/drive/MyDrive/profpatolog/датасет профпатолог часть.xlsx')
df_stock.head()

,ДокументПрохождениеМедосмотра,Клиент,КлиентДатаРождения,КлиентПол,КлиентПолПредставление,ДатаЗавершенияМедосмотра,ТипМедосмотра,ТипМедосмотраПредставление,ПсихОсвидетельствование,Контрагент,...,Оториноларингология3_Аудиометрия1_Значение,Оториноларингология3_Аудиометрия1_ЗначениеПредставление,Оториноларингология1_ЖалобыДополнение_Значение,Оториноларингология1_ЖалобыДополнение_ЗначениеПредставление,Стоматология2_ВП2_Значение,Стоматология2_ВП2_ЗначениеПредставление,Терапия2_ЗаключениеФОГК_Значение,Терапия2_ЗаключениеФОГК_ЗначениеПредставление,Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
0,7f41ca7e-7dac-11ec-80fc-0cc47aab8067,c91e3407-b81f-11e9-80cf-0cc47aab8067,1976-10-24,Женский,Женский,2022-01-25,Периодический,Периодический,False,23c173a7-30d9-11e7-80d8-708bcd7f83cd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2ee1f9ac-83dd-11ec-80fc-0cc47aab8067,1a1a5fea-8b7f-11eb-80e8-0cc47aab8067,1987-11-18,Мужской,Мужской,2022-02-02,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9cbb191e-8e24-11ec-80fe-0cc47aab8067,7fe324e0-fa51-11eb-80f6-0cc47aab8067,1985-09-08,Мужской,Мужской,2022-02-15,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b8a33910-8eec-11ec-80fe-0cc47aab8067,98f9d761-8d65-11ec-80fe-0cc47aab8067,1988-04-25,Мужской,Мужской,2022-02-16,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0bb0d181-938c-11ec-80fe-0cc47aab8067,009aac7d-7bcf-11eb-80e8-0cc47aab8067,1984-04-17,Мужской,Мужской,2022-02-22,Периодический,Периодический,False,a6d559b0-7835-11eb-80e8-0cc47aab8067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_stock.shape

(1982, 4587)

In [4]:
df_cor = df_stock.copy()

In [5]:
# Анализ данных
counts = df_cor.count(axis=0)
unikues = df_cor.nunique(axis=0)
types = df_cor.dtypes
df_x = pd.concat([counts.to_frame(name='count'), unikues.to_frame(name='unikue'), types.to_frame(name='type')], axis= 1 )
df_x

,count,unikue,type
ДокументПрохождениеМедосмотра,1982,1915,object
Клиент,1982,1911,object
КлиентДатаРождения,1982,1797,datetime64[ns]
КлиентПол,1982,2,object
КлиентПолПредставление,1982,2,object
...,...,...,...
Стоматология2_ВП2_ЗначениеПредставление,1,1,object
Терапия2_ЗаключениеФОГК_Значение,1,1,object
Терапия2_ЗаключениеФОГК_ЗначениеПредставление,1,1,object
Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,1,1,object


In [6]:
# Выведем колонки не содержащие данных (бесполезны)
df_x[df_x['count']==0]

,count,unikue,type
ЛабораторныеИсследования1_ХР_Рекомендация2Представление,0,0,float64
ЛабораторныеИсследования1_ХР_РекомендацияСтр,0,0,float64
Рентгенология1_ХР_Рекомендация1,0,0,float64
Рентгенология1_ХР_Рекомендация1Представление,0,0,float64
Рентгенология1_ХР_Рекомендация2,0,0,float64
...,...,...,...
Оториноларингология3_Классификация2Представление1,0,0,float64
Оториноларингология3_Классификация31,0,0,float64
Оториноларингология3_Классификация3Представление1,0,0,float64
Оториноларингология3_Противопоказание1,0,0,float64


In [7]:
#Удалим колонки не имеющие записей
print(f'Размер до - {df_cor.shape}')
df_cor = df_cor.dropna(axis=1, how='all')
print(f'Размер после - {df_cor.shape}')

Размер до - (1982, 4587)
Размер после - (1982, 3703)


In [8]:
# Анализ целевого столбца "ЗаключениеМК"
print(f"заполненных строк - {df_cor['ЗаключениеМК'].count()}, не заполнено - {df_cor['ЗаключениеМК'].isna().sum()}")
print(f"Количество уникальных записей - {df_cor['ЗаключениеМК'].nunique()}")
df_cor['ЗаключениеМК'].unique()

заполненных строк - 1575, не заполнено - 407
Количество уникальных записей - 9


array(['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах',
       'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден',
       'ОграниченноГоден', nan, 'машинист бульдозера', 'Газоспасатель',
       ' Газоспасатель'], dtype=object)

In [9]:
#Удалим строки с ошибками заполнения
count_ind = df_cor.shape[0]
for str1 in ('машинист бульдозера', 'Газоспасатель',' Газоспасатель'):
    df_cor = df_cor.drop(df_cor[df_cor['ЗаключениеМК'] == str1].index)
print(f'Удалено {count_ind - df_cor.shape[0]} строк(и)')
df_cor.shape

Удалено 4 строк(и)


(1978, 3703)

In [10]:
# Строки с незаполненным столбцом выделим в отдельный датафрейм
df_NAN = df_cor[df_cor['ЗаключениеМК'].isna()]
df_NAN.shape

(407, 3703)

In [11]:
 # Удалим строки из основного набора
df_cor = df_cor.drop(df_cor[df_cor['ЗаключениеМК'].isna()].index)
df_cor.shape

(1571, 3703)

In [12]:
# Проверка результата
counts = df_cor.count(axis=0)
unikues = df_cor.nunique(axis=0)
types = df_cor.dtypes
df_x = pd.concat([counts.to_frame(name='count'), unikues.to_frame(name='unikue'), types.to_frame(name='type')], axis= 1 )
df_x

,count,unikue,type
ДокументПрохождениеМедосмотра,1571,1504,object
Клиент,1571,1501,object
КлиентДатаРождения,1571,1426,datetime64[ns]
КлиентПол,1571,2,object
КлиентПолПредставление,1571,2,object
...,...,...,...
Стоматология2_ВП2_ЗначениеПредставление,1,1,object
Терапия2_ЗаключениеФОГК_Значение,1,1,object
Терапия2_ЗаключениеФОГК_ЗначениеПредставление,1,1,object
Офтальмология1_ДвигательныйАппаратГлазДополнение_Значение,1,1,object


In [13]:
# Выведем наиболее заполненные колонки
df_x[df_x['count']>=1570]

,count,unikue,type
ДокументПрохождениеМедосмотра,1571,1504,object
Клиент,1571,1501,object
КлиентДатаРождения,1571,1426,datetime64[ns]
КлиентПол,1571,2,object
КлиентПолПредставление,1571,2,object
ДатаЗавершенияМедосмотра,1571,84,datetime64[ns]
ТипМедосмотра,1571,2,object
ТипМедосмотраПредставление,1571,2,object
ПсихОсвидетельствование,1571,2,bool
Контрагент,1571,142,object


Очевидно на получение конечного результата ("ЗаключениеМК") столбцы с большим количеством уникальных значений ("ДокументПрохожденияМедосмотра", "Клиент") будут иметь небольшое влияние. "КлиентДатаРождения" - может быть полезен для оценки возраста. "ДатаЗавершенияМедосмотра" является лишь датой прохождения медосмотра и значения не имеет 

In [14]:
# Оценка колонки "Профессия"
print(f"незаполненных строк - {df_cor['Профессия'].isna().sum()}")
print(f"Количество уникальных записей - {df_cor['Профессия'].nunique()} из {df_cor['Профессия'].count()}") #
df_cor['Профессия'].unique()[:20]

незаполненных строк - 0
Количество уникальных записей - 557 из 1571


array(['Ведущий специалист по организации обуения', 'Ведущий специалист',
       'Главный менеджер', 'Главный специалист', 'Старший специалист',
       'Начальник отдела', 'Старший инженер', 'Ведущий инженер ',
       'Специалист ', 'Специалист', 'Ведущий специалист ',
       'Мойщик ВС 2 разряда', 'Кладовщик-комплектовщик',
       'Кладовщик комплектовщик', 'Мойщик службы БОВС',
       'Старший кладовщик', 'Комплектовщик',
       'Уборщик производственных помещений', 'Агент-комплектовщик',
       'Уборщик производственных помещений '], dtype=object)

In [15]:
# Проведем колонку через токенайзер
prof_cor = list(df_cor['Профессия'])
tokenizer_prof = Tokenizer(num_words=1000, filters='i1234567890!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)
tokenizer_prof.fit_on_texts(prof_cor)

In [16]:
max(tokenizer_prof.word_index.values())

508

Вывод - для данной колонки целесообразней использовать BagOfWords, к тому же это позволит перекрыть и некоторую дифференциальность заполнения.

In [17]:
# Оценка колонки "ВредныеФакторы"
print(f"незаполненных строк - {df_cor['ВредныеФакторы'].isna().sum()}")
print(f"Количество уникальных записей - {df_cor['ВредныеФакторы'].nunique()} из {df_cor['ВредныеФакторы'].count()}") #
df_cor['ВредныеФакторы'].unique()[:20]

незаполненных строк - 0
Количество уникальных записей - 388 из 1571


array(['11.4', '4.2.5', '11.1', '4.3.1,10,11.1', '1.50,4.2.5,4.1,1.8.1.1',
       '1.30.1,1.34.1,1.37.1', '5.1,6.1,6.2', '5.1,6', '5.1,6.2,6.1',
       '5.1', '6.1,6.2,15', '6.1,6.2',
       '1.8.1.1,1.8.2.1,1.14.2,1.19.1,1.34.1,1.37.1,1.50',
       '1.34.1,1.19.1,1.14.2,1.50,1.37.1,1.8.1.1,1.8.2.1',
       '1.8.1.1,1.8.2.1,1.14.2,1.19.1,1.37.1,1.50,1.34.1',
       '1.8.1.1,1.8.2.1,1.14.2,1.34.1,1.37.1,1.50,1.19.1', '23', '4.4',
       '1.23,1.46,3.4', '1.50,1.34.1,1.37.1'], dtype=object)

Колонка имеет фортат заполнения \d(\.\d)+ перечисленных в некоторых случаях через запятую очевидно что данная колонка может иметь иметь влияния на результат в сочетании с др. колонками. Необходима систематизация.

В список анализуруемых столбцов добавим:
'КлиентДатаРождения',
'КлиентПол',
'ТипМедосмотра',
'ПсихОсвидетельствование',
'Профессия',
'ВредныеФакторы'

In [18]:
df_x[(df_x['count']<1570) & (df_x['count']>=1380)]

,count,unikue,type
Заключение,1559,13,object
ДиспансернаяГруппа,1562,4,object
ДиспансернаяГруппаПредставление,1562,4,object
Терапия1_ДокументПрием,1397,1334,object
Терапия1_ДатаПриема,1397,1334,datetime64[ns]
Терапия1_Салон,1397,2,object
Терапия1_СалонПредставление,1397,2,object
Терапия1_Специализация,1397,1,object
Терапия1_СпециализацияПредставление,1397,1,object
Терапия1_АртериальноеДавление_Значение,1389,92,object


Следующие колонки для анализа:
'ДиспансернаяГруппа', 'Терапия1_АртериальноеДавление_Значение', 
'Терапия1_П_ЗначениеПредставление', 'Терапия1_ГоденНегоден_ЗначениеПредставление', 
'Терапия1_Печень_ЗначениеПредставление', 'Терапия1_Жалобы_ЗначениеПредставление', 
'Терапия1_Анамнез_ЗначениеПредставление', 'Терапия1_ОбщееСостояние_ЗначениеПредставление', 'Терапия1_СВысыпаниями_ЗначениеПредставление', 'Терапия1_Стул_ЗначениеПредставление'

In [20]:
df_x[(df_x['count']<1380) & (df_x['count']>=1370)]

,count,unikue,type
Терапия1_НадЛегкимиДыхание_Значение,1379,1,object
Терапия1_НадЛегкимиДыхание_ЗначениеПредставление,1379,1,object
Терапия1_Язык_Значение,1379,1,object
Терапия1_Язык_ЗначениеПредставление,1379,1,object
Терапия1_ЭОС_Значение,1379,1,object
Терапия1_ЭОС_ЗначениеПредставление,1379,1,object
Терапия1_ТемператураТелаТерапевт_Значение,1373,17,object
Терапия1_ТемператураТелаТерапевт_ЗначениеПредставление,1373,17,object
Терапия1_ФизиологическойОкраски_Значение,1379,1,object
Терапия1_ФизиологическойОкраски_ЗначениеПредставление,1379,1,object


In [21]:
# Оценка колонки "Терапия1_АртериальноеДавление_ЗначениеПредставление"
print(f"незаполненных строк - {df_cor['Терапия1_Печень_ЗначениеПредставление'].isna().sum()}")
print(f"Количество уникальных записей - {df_cor['Терапия1_Печень_ЗначениеПредставление'].nunique()} из {df_cor['Терапия1_Печень_ЗначениеПредставление'].count()}") #
df_cor['Терапия1_Печень_ЗначениеПредставление'].unique()[:50]

незаполненных строк - 177
Количество уникальных записей - 1 из 1394


array(['не увеличена', nan], dtype=object)

In [22]:
df_cor['Терапия1_ТемператураТелаТерапевт_ЗначениеПредставление'].unique()

array([nan, '36,5', '36.6', '36,3', '36,6', '36,7', '36,2', '36,8',
       '36.7', '80', '97', '36', '35', '36.5', '36/6', '164', '36..6',
       '35.6'], dtype=object)

Колонка заполнена неоднородно, имеются явные "вылеты" (80, 97 , 164), нужна фильтрация и обработка, чтобы избежать негативного влияния. Можно отобрать по шаблону: \d\{2}[,./]\d

In [23]:
df_cor['Терапия1_АртериальноеДавление_ЗначениеПредставление'].unique()

array(['140/100', '150/100', '120/80', '120\\80', '110/70', '105/70',
       '120/70', '140/80', '130\\80', '125/85', '133', '120', '100',
       '110', '130', '140', '160', nan, '180', '125', '115', '150', '90',
       '135', '128', '138', '112', '129', '118', '155', '102', '144',
       '107', '105', '141', '182', '145', '131', '123', '193', '121',
       '127', '113', '132', '109', '117', '137', '124', '116', '171',
       '151', '142', '134', '146', '168', '119', '111', '177', '165',
       '122', '153', '159', '143', '147', '148', '167', '136', '139',
       '189', '166', '162', '158', '172', '169', '152', '126', '198',
       '161', '174', '175', '186', '108', '106', '170', '154', '149',
       '173', '157', '130080', '114', '163', '95', '101'], dtype=object)

In [24]:
df_cor['Терапия1_АртериальноеДавление21_ЗначениеПредставление'].unique()

array([ nan,  82.,  80.,  60.,  70.,  90., 110., 120.,  75.,  85.,  84.,
        77.,  72.,  68.,  78.,  83.,  64.,  76.,  74.,  86.,  89.,  87.,
        71.,  88., 112.,  92.,  66., 100.,  62.,  81., 801.,  99.,  91.,
        97., 111., 104., 102.,  95.,  98.,  69.,  96., 105.,  73., 108.,
        63., 101.,  93.,  79.,  67., 107., 106.,  94., 103., 109., 805.,
        65.,  61.,  20.,  58.,  40.,  59.])

In [25]:
# Для примера выведем некотрые строки
df_tmp = df_cor[(df_cor['Терапия1_АртериальноеДавление_ЗначениеПредставление'] == '193') | (df_cor['Терапия1_АртериальноеДавление_ЗначениеПредставление'] == '120\\80') |  (df_cor['Терапия1_АртериальноеДавление_ЗначениеПредставление'] == '140/100')]
df_tmp[['ЗаключениеМК','Терапия1_П_ЗначениеПредставление','Терапия1_АртериальноеДавление_ЗначениеПредставление','Терапия1_АртериальноеДавление21_ЗначениеПредставление']]

,ЗаключениеМК,Терапия1_П_ЗначениеПредставление,Терапия1_АртериальноеДавление_ЗначениеПредставление,Терапия1_АртериальноеДавление21_ЗначениеПредставление
0,Годен,76.0,140/100,NaN
3,Годен,80.0,120\80,NaN
10,ГоденСКоррекциейЗрения,80.0,120\80,NaN
14,Годен,80.0,120\80,NaN
214,Годен,69.0,193,100.0


Вывод: колонка 'Терапия1_АртериальноеДавление_ЗначениеПредставление' должна содержать верхнюю гр. арт. давления, однако в некоторых лучаях заполнена не корректно (иногда сожедит и данные о нижней границе при этом вторая колонка остается незаполненной) при получении данных из неё нужен разбор по элементам.


In [51]:
df_x[(df_x['count']<1370) & (df_x['count']>=1350)][40:]

,count,unikue,type
Неврология1_Ротационный_ЗначениеПредставление,1361,1,object
Неврология1_КрупноРазмашистый_Значение,1361,1,float64
Неврология1_КрупноРазмашистый_ЗначениеПредставление,1361,1,object
Неврология1_СреднеРазмашистый_Значение,1361,1,float64
Неврология1_СреднеРазмашистый_ЗначениеПредставление,1361,1,object
Неврология1_МелкоРазмашистый_Значение,1361,1,float64
Неврология1_МелкоРазмашистый_ЗначениеПредставление,1361,1,object
Неврология1_ПостоянныйВКраевыхОтведениях_Значение,1361,1,float64
Неврология1_ПостоянныйВКраевыхОтведениях_ЗначениеПредставление,1361,1,object
Неврология1_Глотание_Значение,1361,1,object


In [53]:
df_cor['Терапия1_ДиагнозПредставление1'].unique()

array([nan,
       'Гипертензивная [гипертоническая] болезнь с преимущественным поражением сердца без (застойной) сердечной недостаточности',
       'Общий медицинский осмотр', 'Хронический поверхностный гастрит',
       'Атеросклероз',
       'Расстройство вегетативной [автономной] нервной системы неуточненное',
       'Инсулиннезависимый сахарный диабет без осложнений',
       'Другие гастриты',
       'Вегетативная невропатия при эндокринных и метаболических болезнях',
       'Холецистит неуточненный',
       'Язва двенадцатиперстной кишки неуточненная как острая или хроническая, без кровотечения или прободения',
       'Другие хронические панкреатиты',
       'Инсулиннезависимый сахарный диабет с неуточненными осложнениями',
       'Инсулинзависимый сахарный диабет без осложнений',
       'Хронический гастрит неуточненный', 'Миопия', 'Дуоденит',
       'Гастрит неуточненный', 'Перенесенный в прошлом инфаркт миокарда',
       'Хроническая ишемическая болезнь сердца неуточненная',
  

In [54]:
df_x[(df_x['count']<1350) & (df_x['count']>=1300)]

,count,unikue,type
ЛабораторныеИсследования1_ДокументПрием,1300,1238,object
ЛабораторныеИсследования1_ДатаПриема,1300,1238,datetime64[ns]
ЛабораторныеИсследования1_Салон,1300,2,object
ЛабораторныеИсследования1_СалонПредставление,1300,2,object
ЛабораторныеИсследования1_Специализация,1300,1,object
ЛабораторныеИсследования1_СпециализацияПредставление,1300,1,object
ЛабораторныеИсследования1_ОАК_Значение,1300,1,object
ЛабораторныеИсследования1_ОАК_ЗначениеПредставление,1300,1,object
ЛабораторныеИсследования1_Биохимия_Значение,1300,1,object
ЛабораторныеИсследования1_Биохимия_ЗначениеПредставление,1300,1,object


In [67]:
df_cor['Терапия1_Вес_ЗначениеПредставление'].unique()

array([  nan,  68. ,  81. , 112. ,  58. ,  60. ,  90. ,  78. ,  65. ,
       100. ,  95. ,  75. ,  73. ,  80. ,  51. ,  92. ,  72. ,   8. ,
        89. ,  84. ,  62. ,  85. ,  64. ,  67. ,  94. ,  70. ,  61. ,
        49. ,  79. ,  47. ,  55. ,  76. ,  97. , 111. ,  52. ,  83. ,
        44. ,  57. ,  56. ,  54. ,  66. , 110. , 105. ,  86. , 103. ,
        45. , 109.2,  93. , 130. ,  71. ,  48. ,  74. ,  77. ,  69. ,
        82. , 109. , 121. ,  88. ,  59. ,  63. ,  53. ,  96. , 106. ,
       108. , 107. ,  91. ,  98. , 120. ,  87. ,  40. , 115. ,  68.5,
        99. , 114. ,  42. , 104. ,  43. ,  50. ,  53.5, 140. , 102. ,
       137. , 101. , 118. , 128. ,  69.5, 153. , 124. , 150. ,  46. ])

In [61]:
df_cor['Неврология1_Диагноз_ЗначениеПредставление'].unique()

array([nan, '[Z00.0] Общий медицинский осмотр_x000D_\n',
       '[M42.1] Остеохондроз позвоночника у взрослых_x000D_\nКлассификация: Хроническое, ранее выявленное, _x000D_\n',
       '[I69.4] Последствия инсульта, не уточненные как кровоизлияние или инфаркт мозга_x000D_\nКлассификация: Хроническое, ранее выявленное, _x000D_\n',
       '[G12.1] Другие наследственные спинальные мышечные атрофии_x000D_\nКлассификация: Хроническое, ранее выявленное, _x000D_\n',
       '[G25.0] Эссенциальный тремор_x000D_\nКлассификация: Хроническое, ранее выявленное, _x000D_\n',
       '[I67.2] Церебральный атеросклероз_x000D_\n',
       '[E14.8] Сахарный диабет неуточненный с неуточненными осложнениями_x000D_\nКлассификация: Обострение хронического, _x000D_\n',
       '[S51.9] Открытая рана неуточненной части предплечья_x000D_\nКлассификация: Острое, _x000D_\n',
       '[I67.9] Цереброваскулярная болезнь неуточненная_x000D_\nКлассификация: Хроническое, ранее выявленное, _x000D_\n',
       '[I67.2] Церебра

In [107]:
#df_x[(df_x['unikue']<=3) & (df_x['unikue']>1)]
df_x[(df_x['unikue'] == 6)]

,count,unikue,type
ЗаключениеМК,1571,6,object
ЗаключениеМКПредставление,1571,6,object
Офтальмология1_ХР_Рекомендация1,342,6,object
Офтальмология1_ХР_Рекомендация1Представление,342,6,object
Гинекология1_МКБ101,363,6,object
Гинекология1_ДиагнозПредставление1,363,6,object
Стоматология1_ХР_Рекомендация2,17,6,object
Стоматология1_ХР_Рекомендация2Представление,17,6,object
УЗИДиагностика1_Контуры_Значение,593,6,object
УЗИДиагностика1_Структура_Значение,593,6,object


In [110]:
df_cor['Офтальмология1_ХР_Рекомендация1Представление'].unique()

array([nan, 'Допуск с ношением средств оптической коррекции зрения',
       'Использование средств оптической коррекции зрения',
       'Наблюдение офтальмолога по мж', 'Осмотр глазного дна 1 р/год',
       'Обследование и лечение офтальмолога',
       'набдюдение по месту жительства '], dtype=object)